<a href="https://colab.research.google.com/github/m-mehabadi/grad-maker/blob/main/_notebooks/Testing_GradientMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://www.cs.cmu.edu/~pradeepr/convexopt/Lecture_Slides/dual-ascent.pdf

In [21]:
import numpy as np
# import torch
import matplotlib.pyplot as plt

In [22]:

# gd = np.array([
#                [-24, 4],
#                [3, -10],
# ])

# gd = np.random.randn(2, 2)

# gd = np.array([
#                [-0.3131, -0.2785],
#                [ 2.1073,  1.0965]
#                ])

In [23]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')

In [25]:
# def original_gradmaker(grads):
#     alpha = 0.01
#     epsilon = 0.5
#     print("results for the ORIGINAL gradmaker")
#     gd = grads

#     g = np.zeros(2) # the same as the dimension of the space
#     u_ = np.zeros(2) # the same as the number of domains

#     for i in range(20000):
#         u_ = u_+alpha*(epsilon-gd@g)
#         g = 1/2*np.sum(((1+(u_>=0)*u_).reshape(2, 1))*gd, axis=0)

#     print(f'inner product with first vector: {np.dot(g, gd[0])}')
#     print(f'inner product with second vector: {np.dot(g, gd[1])}')

#     # plot vectors
#     plt.plot([0, gd[0,0]], [0, gd[0, 1]], 'go-' , label='g1')
#     plt.plot([0, gd[1,0]], [0, gd[1, 1]], 'ro-',label='g2')
#     plt.plot([0,g[0]], [0,g[1]] ,'bo-' ,label='g*')
#     leg = plt.legend(loc='upper center')
#     plt.show()

# def new_gradmaker(grads):
#     print("results for the NEW gradmaker")
#     gd = grads
#     g = gradient_maker(grads)
#     print(f'inner product with first vector: {np.dot(g, gd[0])}')
#     print(f'inner product with second vector: {np.dot(g, gd[1])}')

#     # plot vectors
#     plt.plot([0, gd[0,0]], [0, gd[0, 1]], 'go-' , label='g1')
#     plt.plot([0, gd[1,0]], [0, gd[1, 1]], 'ro-',label='g2')
#     plt.plot([0,g[0]], [0,g[1]] ,'bo-' ,label='g*')
#     leg = plt.legend(loc='upper center')
#     plt.show()

In [26]:
# vectors = np.random.randn(2, 2)

In [27]:
# print(f'the main vectors:\r\n {vectors}')
# original_gradmaker(vectors)
# new_gradmaker(vectors)

In [65]:
def gradient_maker(domain_grads, epsilon=0.5, alpha=0.01):
    #### TODO: domain_grads must be a tensor itself.
 
    # dgr = torch.stack((domain_grads))
    dgr = domain_grads

    # alpha = 0.01
    # # epsilon = 0.5
    # epsilon = 0.5
    

    number_of_domains, dim = dgr.shape

    # g = torch.zeros(dim).to(device).numpy()
    # u_ = torch.zeros(number_of_domains).to(device).numpy()
    g = np.zeros(dim) #.to(device).numpy()
    u_ = np.zeros(number_of_domains) #.to(device).numpy()
    # prev_g = g
    # current_g = torch.ones(dim).to(device).numpy()
    # it = 0
    # while True :
    #     if np.sum((current_g-prev_g)**2).item()<=0.0001:
    #         break
    dgr_bar = np.sum(dgr, axis=1)/dim
    iter = 0
    # while (not np.abs(np.min(dgr@g)-epsilon)<=0.001) and (iter <= 100):
    # while (not np.min(dgr@g) >= 0.9 * epsilon) and (iter <= 100):
    print(dgr@g)
    while not np.min(dgr@g)/dim >= 0.9 * epsilon:
        # print(dgr@g)
    # for i in range(20000):
        # prev_g = current_g
        u_ = u_ + alpha*(epsilon - (dgr@g))
        # u_ = u_ + alpha*(epsilon - (dgr@g))*(1+1./np.abs(epsilon - (dgr@g)))*(np.abs(epsilon - (dgr@g))>0.001) # this line is to make it converge faster, but didn't work :) don't use.
        g = (1./number_of_domains)*np.sum(((1+(u_>=0)*u_).reshape(number_of_domains, 1))*dgr, axis=0)

        g_bar = np.sum(g)/dim
        E = g_bar.item()*dim*dgr_bar
        inner = dgr@g

        print(f"Iter={iter}: g_bar={g_bar}, E=({E[0]},{E[1]}), Inner=({inner[0]}, {inner[1]}), Condition={np.min(dgr@g)/dim}")
        # current_g = g
        # it = it+1
        iter += 1
    # print("it {}".format(it))    
    # print("Done.")
    return g

### Testcases

Now let's write some test cases to make sure everything is working correctly


In [29]:
# test_num_of_domains = np.random.randint(2,1000)
# test_dim = np.random.randint(test_num_of_domains, int((1+np.random.rand(1).item())*test_num_of_domains))
# test_grads = np.random.randn(test_num_of_domains, test_dim)
# g = gradient_maker(test_grads, epsilon, alpha)
# assert np.abs(np.min(test_grads@g)-epsilon) <= 0.001, "Somewhere is wrong!"
# print(f"in {test_num_of_domains} domains with {test_dim} dimensions a candidate was found!")
# print(f"the minimum inner product is: {np.min(test_grads@g)}")

In [30]:
# g = gradient_maker(test_grads, epsilon, alpha)
# assert np.abs(np.min(test_grads@g)-epsilon) <= 0.001, "Somewhere is wrong!"
# print(f"in {test_num_of_domains} domains with {test_dim} dimensions a candidate was found!")
# print(f"the minimum inner product is: {np.min(test_grads@g)}")

In [67]:
epsilon=100
alpha=0.05

In [68]:
grads = np.random.randn(2, 2000000) + 3
g = gradient_maker(grads, epsilon=epsilon, alpha=alpha)

[0. 0.]
Iter=0: g_bar=18.00307113996874, E=(108058473.48451255,108015240.16267113), Inner=(114061565.84561186, 114028166.36881214), Condition=57.01408318440607
Iter=1: g_bar=3.000511856661458, E=(18009745.5807521,18002540.027111866), Inner=(19010260.974268593, 19004694.394802008), Condition=9.502347197401004
Iter=2: g_bar=3.000511856661458, E=(18009745.5807521,18002540.027111866), Inner=(19010260.974268593, 19004694.394802008), Condition=9.502347197401004
Iter=3: g_bar=3.000511856661458, E=(18009745.5807521,18002540.027111866), Inner=(19010260.974268593, 19004694.394802008), Condition=9.502347197401004
Iter=4: g_bar=3.000511856661458, E=(18009745.5807521,18002540.027111866), Inner=(19010260.974268593, 19004694.394802008), Condition=9.502347197401004
Iter=5: g_bar=3.000511856661458, E=(18009745.5807521,18002540.027111866), Inner=(19010260.974268593, 19004694.394802008), Condition=9.502347197401004
Iter=6: g_bar=3.000511856661458, E=(18009745.5807521,18002540.027111866), Inner=(19010260.

KeyboardInterrupt: ignored